In [1]:
#imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import pathlib

In [2]:
class_names = ['NonDemented', 'Demented']
num_classes = len(class_names)
batch_size = 64
input_shape = (208, 176, 3)
AUTOTUNE = tf.data.experimental.AUTOTUNE

def one_hot_label(image, label):
    label = tf.one_hot(label, num_classes)
    return image, label

In [3]:
train_dir = pathlib.Path('../Project/Alzheimer_s Dataset_binary/train')
test_dir = pathlib.Path('../Project/Alzheimer_s Dataset_binary/test')

train_dataset = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        labels='inferred',
        class_names=class_names,
        batch_size=batch_size,
        image_size=input_shape[0:2])

test_dataset = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        labels='inferred',
        class_names=class_names,
        batch_size=batch_size,
        image_size=input_shape[0:2])

Found 5121 files belonging to 2 classes.
Found 1279 files belonging to 2 classes.


In [4]:
train_dataset = train_dataset.map(one_hot_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(one_hot_label, num_parallel_calls=AUTOTUNE)

In [9]:
vgg = keras.applications.VGG16(
        include_top=False,
        input_shape=input_shape,
        pooling=max)

vgg.trainable = False

model = keras.models.Sequential([
    vgg,
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(num_classes, activation='softmax'),
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 6, 5, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 15360)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                983104    
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 66        
Total params: 15,699,938
Trainable params: 985,250
Non-trainable params: 14,714,688
_________________________________________________________________


In [10]:
epochs = 30
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=["accuracy"])

history = model.fit(train_dataset, epochs=epochs, validation_data=test_dataset)

Epoch 1/30
81/81 [==============================] - 22s 269ms/step - loss: 1.1384 - accuracy: 0.6303 - val_loss: 0.8380 - val_accuracy: 0.6411
Epoch 2/30
81/81 [==============================] - 22s 269ms/step - loss: 0.6180 - accuracy: 0.7290 - val_loss: 0.7022 - val_accuracy: 0.6701
Epoch 3/30
81/81 [==============================] - 22s 268ms/step - loss: 0.4782 - accuracy: 0.7971 - val_loss: 1.0742 - val_accuracy: 0.5825
Epoch 4/30
81/81 [==============================] - 22s 269ms/step - loss: 0.3993 - accuracy: 0.8346 - val_loss: 0.7801 - val_accuracy: 0.6747
Epoch 5/30
81/81 [==============================] - 22s 269ms/step - loss: 0.3184 - accuracy: 0.8752 - val_loss: 0.7822 - val_accuracy: 0.6927
Epoch 6/30
81/81 [==============================] - 22s 271ms/step - loss: 0.2627 - accuracy: 0.9002 - val_loss: 0.7164 - val_accuracy: 0.7146
Epoch 7/30
81/81 [==============================] - 22s 271ms/step - loss: 0.2139 - accuracy: 0.9252 - val_loss: 0.9332 - val_accuracy: 0.6888

In [ ]:
plt.title('VGG16 Transfer Learning Accuracy')

plt.plot(history.history['accuracy'], label = "training acc.")
plt.plot(history.history['val_accuracy'], label = "validation acc.")
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training acc.', 'validation acc.'], loc='upper left')

plt.savefig("../Project/Result2/VGG16_MODEL_ACCURACY_RMSprop")
plt.show()
